In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

#NEO4J_URI = os.environ.get("NEO4J_URI")
NEO4J_URI = "bolt://localhost:7687"
#NEO4J_USERNAME = os.environ.get("NEO4J_USERNAME")
#NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")

In [7]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(NEO4J_URI)

In [30]:
import csv
with open("data/nr-stations-all.csv", "r", encoding="utf-8-sig") as file:
    reader = csv.DictReader(file)
    stations = list(reader)

print(f"Loaded {len(stations)} stations")

Loaded 2593 stations


In [32]:
# Load stations as nodes
gds.run_cypher('match (n) detach delete(n)')
for station in stations:
    gds.run_cypher(f'CREATE (:Station {{name: "{station["name"]}", crs: "{station["crs"]}"}});')

In [27]:
import csv
with open("data/nr-station-links.csv", "r", encoding="utf-8-sig") as file:
    reader = csv.DictReader(file)
    links = list(reader)

print(f"Loaded {len(links)} links")

Loaded 5802 links


In [34]:
for link in links:
    gds.run_cypher(
        f"""
            MATCH (st_from: Station), (st_to: Station)
            WHERE st_from.crs="{link["from"]}" and st_to.crs="{link["to"]}"
            MERGE (st_from)-[:TRACK {{distance: {round(float(link["distance"]), 2)}}}]->(st_to)
        """)